In [2]:
from pycaret.regression import load_model, predict_model
import pandas as pd
from ax.service.managed_loop import optimize

In [3]:
# 1. pycaret에서 기존에 구축한 모델 불러오기
model_R = load_model('et_saved_220525_test3')
model_P = load_model('et2_saved_220525_test3')
model_V = load_model('et3_saved_220525_test3')
# Load the test data from CSV into a DataFrame
test_df = pd.read_csv('Test_data_ML-R9_220516_test3_MW.csv', encoding='cp949')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [ ]:
# 2. 모델이 잘 불러왔는지, 작동하는지 테스트 (test 데이터 csv를 이용)
# test_results = predict_model(model_R, data=test_df)
# print(test_results.head())
# test_results = predict_model(model_P, data=test_df)
# print(test_results.head())
# test_results = predict_model(model_V, data=test_df)
# print(test_results.head())

In [4]:
input_df = pd.read_csv('Bays_data_initial 10_251007.csv', encoding='cp949')

In [15]:
# 3. ax를 이용하여 해당 모델의 output 최적화 수행
# 최적화할 파라미터 범위 정의 (예시: Feed rate, Rinse rate, Pad (bar), Pde (bar))
def pycaret_predict(params):
    # 컬럼 이름에 따라 파라메터 매핑
    param_map = {
        "Feed_rate_SLPM": "Feed rate (SLPM)",
        "Rinse_rate_SLPM": "Rinse rate (SLPM)",
        "Pad_bar": "Pad (bar)",
        "Pde_bar": "Pde (bar)"
    }
    # 평균값을 기준으로 시작점 수립
    updated_base = base.copy()
    # 제안받은 파라메터 업데이트
    for k, v in params.items():
        updated_base[param_map[k]] = v
    # 모델 데이터 입력 전 배열 맞추기
    input_row_updated = {col: updated_base[col] for col in columns}
    input_df_pred = pd.DataFrame([input_row_updated])
    result = predict_model(model_R, data=input_df_pred)
    # 모델 예측값을 출력
    return result['prediction_label'].iloc[0]

best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "Feed_rate_SLPM", "type": "range", "bounds": [1.0, 10.0]},
        {"name": "Rinse_rate_SLPM", "type": "range", "bounds": [0.5, 2.4]},
        {"name": "Pad_bar", "type": "range", "bounds": [2.5, 6.4]},
        {"name": "Pde_bar", "type": "range", "bounds": [0.05, 0.15]},
    ],
    evaluation_function=pycaret_predict, # 평가 함수로 pycaret 모델 예측 함수 사용
    minimize=False, # True if minimizing the objective, False if maximizing
    total_trials=20, # 총 시도 횟수
)

print("Best parameters found:", best_parameters)
print("Best objective value:", values)

[INFO 10-07 16:34:28] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter Feed_rate_SLPM. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-07 16:34:28] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter Rinse_rate_SLPM. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-07 16:34:28] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter Pad_bar. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-07 16:34:28] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter Pde_bar. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or

[INFO 10-07 16:34:29] ax.service.managed_loop: Running optimization trial 2...
[INFO 10-07 16:34:29] ax.service.managed_loop: Running optimization trial 3...
[INFO 10-07 16:34:29] ax.service.managed_loop: Running optimization trial 4...
[INFO 10-07 16:34:29] ax.service.managed_loop: Running optimization trial 5...
[INFO 10-07 16:34:30] ax.service.managed_loop: Running optimization trial 6...
[INFO 10-07 16:34:30] ax.service.managed_loop: Running optimization trial 7...
[INFO 10-07 16:34:30] ax.service.managed_loop: Running optimization trial 8...
[INFO 10-07 16:34:31] ax.service.managed_loop: Running optimization trial 9...
[INFO 10-07 16:34:37] ax.service.managed_loop: Running optimization trial 10...
[INFO 10-07 16:34:43] ax.service.managed_loop: Running optimization trial 11...
[INFO 10-07 16:34:48] ax.service.managed_loop: Running optimization trial 12...
[INFO 10-07 16:34:55] ax.service.managed_loop: Running optimization trial 13...
[INFO 10-07 16:35:02] ax.service.managed_loop: R

Best parameters found: {'Feed_rate_SLPM': 1.6799610048132445, 'Rinse_rate_SLPM': 0.8444925962646864, 'Pad_bar': 6.4, 'Pde_bar': 0.05}
Best objective value: ({'objective': 100.37210138677126}, {'objective': {'objective': 0.4055991900150664}})
